# Experiment 6

This experiment aims to explore the number of unique links that the swarm is able to explore in a given number of turns. This way, the swarm can be compared to other networks that when comparing percentage explored fails as a means for comparison. This is because percentage explored is a function of the size of the network, and thus, a swarm with a smaller network will always have a higher percentage explored in a given number of turns.

In [1]:
## Imports

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import networkx as nx

import logging

from src.simulation import Simulation
from src.network import Network
from src.render import Render

logging.disable(logging.CRITICAL)

In [2]:
network_file_1 = "networks/Net3.inp"
network_file_2 = "networks/Net6.inp"
network_file_3 = "networks/250701 K709vs2-Export.inp"

env1 = Network(network_file_1)
env2 = Network(network_file_2)
env3 = Network(network_file_3)

g_env1 = env1.water_network_model.to_graph().to_undirected()
g_env2 = env2.water_network_model.to_graph().to_undirected()
g_env3 = env3.water_network_model.to_graph().to_undirected()

d_env1 = g_env1.degree
d_env2 = g_env2.degree
d_env3 = g_env3.degree

env1_start_pool = [node for node, degree in d_env1 if degree == 1]
env2_start_pool = [node for node, degree in d_env2 if degree == 1]
env3_start_pool = [node for node, degree in d_env3 if degree == 1]

env1_num_links = env1.graph_num_links
env2_num_links = env2.graph_num_links
env3_num_links = env3.graph_num_links

print("Start Pool - Net3: ", env1_start_pool)
print("Number of Start Nodes - Net3: ", len(env1_start_pool))
print("Number of Nodes - Net3: ", env1.graph_num_nodes)
print("Number of Links - Net3: ", env1.graph_num_links)

print("Start Pool - Net6: ", env2_start_pool)
print("Number of Start Nodes - Net6: ", len(env2_start_pool))
print("Number of Nodes - Net6: ", env2.graph_num_nodes)
print("Number of Links - Net6: ", env2.graph_num_links)

print("Start Pool - 250701 K709vs2-Export: ", env3_start_pool)
print("Number of Start Nodes - 250701 K709vs2-Export: ", len(env3_start_pool))
print("Number of Nodes - 250701 K709vs2-Export: ", env3.graph_num_nodes)
print("Number of Links - 250701 K709vs2-Export: ", env3.graph_num_links)

Start Pool - Net3:  ['15', '35', '131', '166', '167', '203', '219', '225', '231', '243', '253', 'River', 'Lake', '1', '2', '3']
Number of Start Nodes - Net3:  16
Number of Nodes - Net3:  97
Number of Links - Net3:  119
Start Pool - Net6:  ['JUNCTION-12', 'JUNCTION-16', 'JUNCTION-20', 'JUNCTION-21', 'JUNCTION-38', 'JUNCTION-39', 'JUNCTION-40', 'JUNCTION-41', 'JUNCTION-45', 'JUNCTION-53', 'JUNCTION-74', 'JUNCTION-75', 'JUNCTION-81', 'JUNCTION-88', 'JUNCTION-96', 'JUNCTION-101', 'JUNCTION-113', 'JUNCTION-116', 'JUNCTION-138', 'JUNCTION-147', 'JUNCTION-150', 'JUNCTION-161', 'JUNCTION-178', 'JUNCTION-189', 'JUNCTION-204', 'JUNCTION-210', 'JUNCTION-218', 'JUNCTION-233', 'JUNCTION-234', 'JUNCTION-235', 'JUNCTION-247', 'JUNCTION-252', 'JUNCTION-255', 'JUNCTION-264', 'JUNCTION-266', 'JUNCTION-272', 'JUNCTION-276', 'JUNCTION-305', 'JUNCTION-311', 'JUNCTION-313', 'JUNCTION-332', 'JUNCTION-334', 'JUNCTION-341', 'JUNCTION-343', 'JUNCTION-346', 'JUNCTION-350', 'JUNCTION-351', 'JUNCTION-360', 'JUNCTI

In [3]:
def run_simulation_batch(env, num_agents, start_nodes, filepath, max_turns=100):
    # Run the simulations for no swarm control
    path = f'{filepath}/NoSwarm'
    simulations_1 = []
    for node in start_nodes:
        print("Starting No Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=False, start_positions=[node], filepath=path)
        simulations_1.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)
    yield simulations_1
    
    # Run the simulations for naive swarm control
    path = f'{filepath}/NaiveSwarm'
    swarm_config = {'swarm': True, 'swarm_type': 'naive'}
    simulations_2 = []
    for node in start_nodes:
        print("Starting Naive Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=True, swarm_config=swarm_config, start_positions=[node], filepath=path)
        simulations_2.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)
    yield simulations_2
    
    # Run the simulations for informed mean swarm control
    path = f'{filepath}/InformedMeanSwarm'
    swarm_config = {'swarm': True, 'swarm_type': 'informed', 'allocation_threshold': 'mean'}
    simulations_3 = []
    for node in start_nodes:
        print("Starting Informed Mean Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=True, swarm_config=swarm_config, start_positions=[node], filepath=path)
        simulations_3.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)    
    yield simulations_3
    
    # Run the simulations for informed median swarm control
    path = f'{filepath}/InformedMedianSwarm'
    swarm_config = {'swarm': True, 'swarm_type': 'informed', 'allocation_threshold': 'median'}
    simulations_4 = []
    for node in start_nodes:
        print("Starting Informed Median Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=True, swarm_config=swarm_config, start_positions=[node], filepath=path)
        simulations_4.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)
    yield simulations_4

## Simulation 6.1

This set of simulations will focus on `env1`. There will be 10 agents, and the number of turns will be 100. In each simulation, all agents will begin at the same start node though to reduce the influence of the starting position on the final resuslts, the simulation will be run 15 times with the first 15 nodes in the starting node pool as the agent starting positions. The number of unique links explored will be recorded for each simulation.

In [4]:
start_nodes = env1_start_pool[:15]
num_agents = 10
max_turns = 100
filepath = "notable-results/Experiment-6/Net3"

print("Start Nodes: ", start_nodes)

Start Nodes:  ['15', '35', '131', '166', '167', '203', '219', '225', '231', '243', '253', 'River', 'Lake', '1', '2']


In [9]:
simulations_6 = run_simulation_batch(env1, num_agents, start_nodes, filepath, max_turns=max_turns)
no_swarm_env3 = next(simulations_6)
naive_swarm_env3 = next(simulations_6)
informed_mean_swarm_env3 = next(simulations_6)
informed_median_swarm_env3 = next(simulations_6)

Starting No Swarm Simulation from Start Node:  15
Starting No Swarm Simulation from Start Node:  35
Starting No Swarm Simulation from Start Node:  131
Starting No Swarm Simulation from Start Node:  166
Starting No Swarm Simulation from Start Node:  167
Starting No Swarm Simulation from Start Node:  203
Starting No Swarm Simulation from Start Node:  219
Starting No Swarm Simulation from Start Node:  225
Starting No Swarm Simulation from Start Node:  231
Starting No Swarm Simulation from Start Node:  243
Starting No Swarm Simulation from Start Node:  253
Starting No Swarm Simulation from Start Node:  River
Starting No Swarm Simulation from Start Node:  Lake
Starting No Swarm Simulation from Start Node:  1
Starting No Swarm Simulation from Start Node:  2
Starting Naive Swarm Simulation from Start Node:  15
Starting Naive Swarm Simulation from Start Node:  35
Starting Naive Swarm Simulation from Start Node:  131
Starting Naive Swarm Simulation from Start Node:  166
Starting Naive Swarm Sim

### Simulation 6.1.1

- `env`: Net3
- `num_agents`: 10
- `swarm`: False
- `swarm_config`: None
- `start_node`: 15 nodes from start node pool
- `max_turns`: 100
- `filepath`: notable-results/Experiment6/Env1/NoSwarm

In [ ]:
simulations_6_1_1 = []

for start_node in start_nodes:
    print("Beginning simulation with start node: ", start_node)
    sim = Simulation(environment=env1, num_agents=num_agents, swarm=False, start_positions=[start_node], filepath="notable-results/Experiment-6/Env1/NoSwarm")
    simulations_6_1_1.append((start_node, sim.path_to_results_directory))
    sim.run()

### Simulation 6.1.2

- `env`: Net3
- `num_agents`: 10
- `swarm`: True
- `swarm_config`: naive
- `start_node`: 15 nodes from start node pool
- `max_turns`: 100
- `filepath`: notable-results/Experiment6/Env1/NaiveSwarm

In [ ]:
simulations_6_1_2 = []

swarm_config = {'swarm': True, 'swarm_type': 'naive'}

for start_node in start_nodes:
    print("Beginning simulation with start node: ", start_node)
    sim = Simulation(environment=env1, num_agents=num_agents, swarm=True, swarm_config=swarm_config, start_positions=[start_node], filepath="notable-results/Experiment-6/Env1/NaiveSwarm")
    simulations_6_1_2.append((start_node, sim.path_to_results_directory))
    sim.run()

### Simulation 6.1.3

- `env`: Net3
- `num_agents`: 10
- `swarm`: True
- `swarm_config`: informed, mean
- `start_node`: 15 nodes from start node pool
- `max_turns`: 100
- `filepath`: notable-results/Experiment6/Env1/InformedMeanSwarm

In [ ]:
simulations_6_1_3 = []

swarm_config = {'swarm': True, 'swarm_type': 'informed', 'allocation_threshold': 'mean'}

for start_node in start_nodes:
    print("Beginning simulation with start node: ", start_node)
    sim = Simulation(environment=env1, num_agents=num_agents, swarm=True, swarm_config=swarm_config, start_positions=[start_node], filepath="notable-results/Experiment-6/Env1/InformedMeanSwarm")
    simulations_6_1_3.append((start_node, sim.path_to_results_directory))
    sim.run()

### Simulation 6.1.4

- `env`: Net3
- `num_agents`: 10
- `swarm`: True
- `swarm_config`: informed, median
- `start_node`: 15 nodes from start node pool
- `max_turns`: 100
- `filepath`: notable-results/Experiment6/Env1/InformedMedianSwarm

In [ ]:
simulations_6_1_4 = []

swarm_config = {'swarm': True, 'swarm_type': 'informed', 'allocation_threshold': 'median'}

for start_node in start_nodes:
    print("Beginning simulation with start node: ", start_node)
    sim = Simulation(environment=env1, num_agents=num_agents, swarm=True, swarm_config=swarm_config, start_positions=[start_node], filepath="notable-results/Experiment-6/Env1/InformedMedianSwarm")
    simulations_6_1_4.append((start_node, sim.path_to_results_directory))
    sim.run()